In [104]:
import pandas as pd
import itertools
import numpy as np

In [105]:
def calcola_percentuali(variazioni, ordine_di_partenza):
    # Creazione di tutte le possibili combinazioni di + e -
    combinazioni = [''.join(p) for p in itertools.product(['+', '-'], repeat=ordine_di_partenza)]
    
    # Inizializzazione di una lista vuota per raccogliere le righe
    righe = []
    
    # Calcolo delle percentuali per ogni combinazione
    for combinazione in combinazioni:
        riga = {'Combinazione': combinazione}
        ultimo_simbolo = combinazione[-1]
        simbolo_opposto = '+' if ultimo_simbolo == '-' else '-'
        indici = [i for i in range(len(variazioni) - len(combinazione)) if variazioni[i:i+len(combinazione)]==combinazione]
        
        # Inizializzazione della somma delle percentuali
        somma_percentuali = 0
        
        # Calcolo della percentuale per 0 volte (rappresenta la percentuale di volte in cui la riga in questione ha subito dopo il simbolo opposto all'ultimo simbolo, cioè + se finisce con - e - se finisce con +)
        conteggio_0 = sum([1 for indice in indici if variazioni[indice+len(combinazione)]==simbolo_opposto])
        percentuale_0 = conteggio_0 / len(indici) if indici else 0
        somma_percentuali += percentuale_0
        riga['Prosecuzione 0 volte'] = round(percentuale_0, 3)
        
        i = 1
        while somma_percentuali < 1 and i < len(variazioni) - len(combinazione):
            conteggio = sum([1 for indice in indici if indice+len(combinazione)+i < len(variazioni) and variazioni[indice+len(combinazione):indice+len(combinazione)+i]==ultimo_simbolo*i and variazioni[indice+len(combinazione)+i]==simbolo_opposto])
            percentuale = conteggio / len(indici) if indici else 0
            somma_percentuali += percentuale
            if i == 1:
                colonna = f'Prosecuzione {i} volta'
            else:
                colonna = f'Prosecuzione {i} volte'
            riga[colonna] = round(percentuale, 3)
            
            # Incremento i per la prossima iterazione
            i += 1
        
        # Aggiunta della riga alla lista di righe
        righe.append(pd.DataFrame([riga]))
    
    # Concatenazione di tutte le righe in un unico DataFrame
    df = pd.concat(righe, ignore_index=True)
    
    # Impostazione dell'indice del DataFrame
    df.set_index('Combinazione', inplace=True)

    # Metto a 0 i valori NaN
    df.fillna(0, inplace=True)

    # Elimino le colonne che contengono solo zeri
    df = df.loc[:, (df != 0).any(axis=0)]
    return df

In [106]:
dati = pd.read_csv('Bitcoin (€) da yfinance dal 17-09-2014 al 24-04-2024.csv')
dati = dati[['Date', 'Close']]
dati = dati.rename(columns = {'Date':'Timestamp', 'Close': 'Price'})
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d')
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d')
dati

,Timestamp,Price
0,2014-09-17,355.957367
1,2014-09-18,328.539368
2,2014-09-19,307.761139
3,2014-09-20,318.758972
4,2014-09-21,310.632446
...,...,...
3503,2024-04-20,59876.710938
3504,2024-04-21,60956.074219
3505,2024-04-22,60919.242188
3506,2024-04-23,62729.296875


In [107]:
data_inizio = '2020-03-01'
data_fine = '2021-03-01'
indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
dati

,Timestamp,Price
0,2020-03-01,7748.164551
1,2020-03-02,7959.358398
2,2020-03-03,7862.485352
3,2020-03-04,7861.055176
4,2020-03-05,8089.786133
...,...,...
361,2021-02-25,38735.164062
362,2021-02-26,38385.910156
363,2021-02-27,38260.480469
364,2021-02-28,37332.503906


In [108]:
dati['Daily_return'] = dati['Price'].pct_change()
dati['State'] = np.where(dati['Daily_return'] >= 0 & dati['Daily_return'].notna(), "+", np.where(dati['Daily_return'].notna(), "-", float(np.nan)))
dati

,Timestamp,Price,Daily_return,State
0,2020-03-01,7748.164551,NaN,nan
1,2020-03-02,7959.358398,0.027257,+
2,2020-03-03,7862.485352,-0.012171,-
3,2020-03-04,7861.055176,-0.000182,-
4,2020-03-05,8089.786133,0.029097,+
...,...,...,...,...
361,2021-02-25,38735.164062,-0.051641,-
362,2021-02-26,38385.910156,-0.009016,-
363,2021-02-27,38260.480469,-0.003268,-
364,2021-02-28,37332.503906,-0.024254,-


In [109]:
dati = dati.dropna().reset_index()
dati

,index,Timestamp,Price,Daily_return,State
0,1,2020-03-02,7959.358398,0.027257,+
1,2,2020-03-03,7862.485352,-0.012171,-
2,3,2020-03-04,7861.055176,-0.000182,-
3,4,2020-03-05,8089.786133,0.029097,+
4,5,2020-03-06,8082.739746,-0.000871,-
...,...,...,...,...,...
360,361,2021-02-25,38735.164062,-0.051641,-
361,362,2021-02-26,38385.910156,-0.009016,-
362,363,2021-02-27,38260.480469,-0.003268,-
363,364,2021-02-28,37332.503906,-0.024254,-


In [110]:
"".join(dati['State'].to_list())

'+--+----++-+-+-+++---++-----++++-+-+-+---+---+-+--+++++++++-+++++++----+++-++----++-+-++-+-+-++--++-+-+---+---+-+-----++-+--+-+-+---+-----++-+++-+++-+-++-+-++-+-+-++-+++--+-+-+-+-+-+-+--+-++-++++-+++-++--+-+-++-+----++-++++++-++-++++++-+-++-+++--+++--+--++---+++-++-++---+++-++-++--+--++++++++--+-+++-+++++++-++++----++---+---+-+++-++--+++-++-++-+--+-++-+++--+----+'

In [111]:
ordine_di_partenza = 2

df = calcola_percentuali("".join(dati['State'].to_list()), ordine_di_partenza)
df
# La tabella rappresenta la percentuale di volte che c'è stata una prosecuzione dell'ultimo simbolo della riga in questione per un certo numero di volte (per poi cambiare segno); ad esempio, alla riga ++++ e colonna "Prosecuzione 0 volte" ho la percentuale di volte che c'è stato un ++++ e poi -, cioè subito un cambio, mentre la colonna "Prosecuzione 1 volta" rappresenta il numero di volte che c'è stato un ++++ e poi un +-, mentre "Prosecuzione 2 volte" indica la % di volte che c'è stato un ++++ e poi ++-; con una riga che finisce con il - è l'opposto: ad esempio, +++- avrà nella colonna 0 la % di volte che dopo +++- c'è stato un +, la colonna 1 avrà la % di volte che dopo un +++- c'è stato un -+ e così via

,Prosecuzione 0 volte,Prosecuzione 1 volta,Prosecuzione 2 volte,Prosecuzione 3 volte,Prosecuzione 4 volte,Prosecuzione 5 volte,Prosecuzione 6 volte,Prosecuzione 7 volte,Prosecuzione 8 volte
Combinazione,,,,,,,,,
++,0.500,0.225,0.088,0.059,0.059,0.039,0.020,0.01,0.00
+-,0.643,0.163,0.102,0.061,0.031,0.000,0.000,0.00,0.00
-+,0.474,0.289,0.144,0.031,0.000,0.021,0.021,0.01,0.01
--,0.530,0.288,0.136,0.045,0.000,0.000,0.000,0.00,0.00
